In [11]:
import re
import pandas as pd
import csv
from io import StringIO
import os

def sql_to_dataframes(sql_file_path):
    with open(sql_file_path, 'r', encoding='utf-8') as f:
        sql = f.read()

    tables = {}

    # Step 1: Parse CREATE TABLE statements to get column order
    create_table_matches = re.findall(
        r'CREATE TABLE [`\[]?(\w+)[`\]]?\s*\((.*?)\);',
        sql, re.DOTALL | re.IGNORECASE
    )

    for table_name, columns_block in create_table_matches:
        columns = []
        lines = [line.strip().rstrip(',') for line in columns_block.strip().splitlines()]
        for line in lines:
            if line.upper().startswith(('PRIMARY KEY', 'UNIQUE', 'KEY', 'CONSTRAINT', 'FOREIGN')):
                continue
            match = re.match(r'[`[]?(\w+)[`\]]?\s', line)
            if match:
                columns.append(match.group(1))
        tables[table_name] = {'columns': columns, 'rows': []}

    # Step 2: Parse INSERT INTO with single or multiple VALUES
    insert_statements = re.findall(
        r'INSERT INTO [`\[]?(\w+)[`\]]?\s*\((.*?)\)\s*VALUES\s*(.*?);',
        sql, re.DOTALL | re.IGNORECASE
    )

    for table_name, col_block, values_block in insert_statements:
        if table_name not in tables:
            print(f"⚠️ Skipping unknown table {table_name}")
            continue

        insert_columns = [c.strip(' `[]') for c in col_block.split(',')]

        # Extract individual value tuples
        value_tuples = re.findall(r'\((.*?)\)', values_block, re.DOTALL)
        for value_group in value_tuples:
            cleaned = value_group.replace('\n', ' ').replace('\r', ' ').strip()
            reader = csv.reader([cleaned], skipinitialspace=True)
            try:
                values = next(reader)
            except Exception as e:
                print(f"❌ Error parsing row in {table_name}: {e}")
                continue

            # Map to full column order
            full_row = [''] * len(tables[table_name]['columns'])
            for col, val in zip(insert_columns, values):
                if col in tables[table_name]['columns']:
                    idx = tables[table_name]['columns'].index(col)
                    full_row[idx] = val
            tables[table_name]['rows'].append(full_row)

    # Step 3: Convert to pandas DataFrames
    dataframes = {}
    for table_name, data in tables.items():
        df = pd.DataFrame(data['rows'], columns=data['columns'])
        dataframes[table_name] = df
        print(f"✅ Loaded {table_name}: {df.shape[0]} rows")

    return dataframes

def save_dataframes_to_csv(dataframes, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for table_name, df in dataframes.items():
        output_path = os.path.join(output_dir, f"{table_name}.csv")
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✅ Saved {table_name}.csv with {len(df)} rows")

    print(f"\n🎉 All CSVs saved to: {output_dir}")

In [13]:
# Step 1: Load SQL dump into DataFrames
dataframes = sql_to_dataframes("input/CLERUS_v1.sql")

# Step 2: Save each DataFrame to CSV
save_dataframes_to_csv(dataframes, "output/csv_tables")

✅ Loaded 01_clerus_bio: 12628 rows
✅ Loaded 02_drc_org: 14489 rows
✅ Loaded 03_clerus_exam: 0 rows
✅ Loaded 11_clerus_alt_name: 12595 rows
✅ Loaded 12_clerus_role: 34165 rows
✅ Loaded 21_birth_alt_place: 0 rows
✅ Loaded 22_baptism_alt_place: 0 rows
✅ Loaded 23_death_alt_place: 0 rows
✅ Loaded 24_burried_alt_place: 0 rows
✅ Loaded 25_role_alt_place: 0 rows
✅ Loaded 26_residence_alt_place: 0 rows
✅ Saved 01_clerus_bio.csv with 12628 rows
✅ Saved 02_drc_org.csv with 14489 rows
✅ Saved 03_clerus_exam.csv with 0 rows
✅ Saved 11_clerus_alt_name.csv with 12595 rows
✅ Saved 12_clerus_role.csv with 34165 rows
✅ Saved 21_birth_alt_place.csv with 0 rows
✅ Saved 22_baptism_alt_place.csv with 0 rows
✅ Saved 23_death_alt_place.csv with 0 rows
✅ Saved 24_burried_alt_place.csv with 0 rows
✅ Saved 25_role_alt_place.csv with 0 rows
✅ Saved 26_residence_alt_place.csv with 0 rows

🎉 All CSVs saved to: output/csv_tables
